### <p style="text-align: right;"> &#9989; **Group 1 (GoGreen)** </p>
#### <p style="text-align: right;"> &#9989; Beth, Zhongjie, McKenna, Erik</p>

# Module 1



## How do SNPs relate to gene transcription?

We will work with maize. Last updated 2020/01/23

Jupyter by default converts `tabs` into `spaces` which is good.

First, import the usual libraries
- `math`: basic math operations
- `os`: enable file manipulation with the OS
- `sys`: enable interaction with commandline
- `glob`: more variable manipulation
- `matplotlib.pyplot`: default plotter (I personally like ggplot waaaaay better. E)
    - `inline`: so that plots are shown in the notebook
- `seaborn`: nicer plots
- `numpy`: all number cruching done here
- `pandas`: data wrangling

In [1]:
import math  
import os   
import glob 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd


Define some functions. Let's see for how long can we keep tidy commented code.

In [2]:
def dot_to_int(char_in, substitute='.', value=0.0):
    """
    Substitute a string for a given value if such string is of interest
    
    Parameters
    ----------
    char_in : string
         possible char to be substituted
    
    substitute : string
         if the given input matches this object, substitute
    
    value : int
         if applicable, substitute the input with this value
    
    Returns
    -------
    value : if input matches substitute, else returns unchanged input
    """
    if(char_in == substitute):
        return value
    else:
        return char_in
    
def numMissing(row, start=10, missing='.'):
    foo = row[start:] == missing
    return(foo.sum())


def numData(row, data_cols=['Total Calls', 'Missing Data']):
    return row[data_cols].sum()  

_Sidenote:_ We can print whatever text was written between the `"""` above.

In [3]:
print(dot_to_int.__doc__)


    Substitute a string for a given value if such string is of interest
    
    Parameters
    ----------
    char_in : string
         possible char to be substituted
    
    substitute : string
         if the given input matches this object, substitute
    
    value : int
         if applicable, substitute the input with this value
    
    Returns
    -------
    value : if input matches substitute, else returns unchanged input
    


Make sure to write the right path to the files

In [4]:
# Path to the folder where all the raw data is located
src = '/home/ejam/documents/css893/raw_data/'

# Path where we will save outputs
dst = '/home/ejam/documents/css893/output/'

# Matrix with raw SNP data
file_snp_raw = 'B73_plus_RTAs_snp_matrix_995785.txt'

# Matrix with imputed SNP data
file_snp_imp = 'B73_plus_RTAs_snp_matrix_imputed/widiv_942g_979873SNPs_imputed_filteredGenos_withRTA_AGPv4.hmp.txt'

# Matrix with gene expression (FPKM)
file_gen = '942_FPKM_B73_genes_w_feature.txt'

Load the SNP matrix. __Loading the full SNP matrix consumes about 16Gb!!!__ You may need more to do some basic data wrangling. I need way more memory to do fancier stuff 😿.
If you don't have at least 32GB of RAM in your machine, you can simply load the first 1000 SNPs. This will give you a sense of how the data looks like.

In [5]:
# BEWARE : Line below consumes 16Gb of RAM at some point. 
# Trying to do some basic wrangling (i.e. masking) may consume additional 8Gb
# More sophisticated wrangling (apply the same function to all rows) may consume much more RAM

# We already know that beyond SNP 899784, the information is irrelevant (contigs and RTAs)
# Set clean = False to see how such cutoff was computed
clean = True

if clean:
    snp_matrix_imputed = pd.read_table(src + file_snp_imp, dtype={'chrom': int}, nrows=899784)
else:
    snp_matrix_imputed = pd.read_table(src + file_snp_imp, dtype={'chrom': int})

# Run the commented line below instead if you don't have much RAM
#snp_matrix_imputed = pd.read_table(src + file_snp, dtype={'chromosome': str}, nrows=1000)
snp_matrix_raw = pd.read_table(src + file_snp_raw, dtype={'chromosome': str}, nrows=1000)

Get a sense of how the data is arranged in the raw SNP matrix. The full matrix has 995785 rows × 952 columns.
- Each row is a SNP
- The first ten columns are metadata
- The other 942 correspond to different cultivars.

The column names for cultivars is found in `B73_plus_RTAs_snp_matrix_imputed/widiv_942g_979873SNPs_imputed_filteredGenos_withRTA_AGPv4.hmp.txt`

You may notice that `Missing Data` + `Total Calls` = 959 for every SNP. However, we only have information from 942 cultivars. That is, the data suggests there are 17 samples are missing. From Gage _et al_ 2019:

> Based on their location in the tree and known pedigree 
information, the inbred line Mo8W was removed from the 
SNP and fragments per kb of exon model per million mapped
reads (FPKM) matrix files. An additional 16 inbred lines
were also removed from the SNP and FPKM files, resulting
in a final set of 942 inbreds, referred to as the 
WiDiv-942 (Mazaheri et al.,2019), which were analyzed in 
this study.

In [6]:
snp_matrix_raw

,chromosome,position,reference_allele,Allele Counts,Missing Data,Total Calls,Percent A,Percent T,Percent C,Percent G,...,NC328,NC326,PHV53,DKIBC2,A641,WIL900,Va22,E8501,PHP85,Oh43
0,B73V4_ctg102,36353,A,A(0)T(0)C(0)G(264),695,264,.,.,.,100,...,.,.,G,G,G,.,.,G,G,G
1,B73V4_ctg10,292327,T,A(0)T(744)C(0)G(105),110,849,.,87.6325088339223,.,12.3674911660777,...,T,T,.,T,T,.,G,T,T,G
2,B73V4_ctg10,292384,G,A(2)T(0)C(0)G(906),51,908,0.220264317180617,.,.,99.7797356828194,...,G,G,G,G,G,G,G,G,G,G
3,B73V4_ctg10,292386,G,A(107)T(0)C(0)G(794),58,901,11.8756936736959,.,.,88.1243063263041,...,G,G,G,G,G,G,A,G,G,A
4,B73V4_ctg10,292402,A,A(912)T(1)C(0)G(0),46,913,99.8904709748083,0.109529025191676,.,.,...,A,A,A,A,A,A,A,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,B73V4_ctg129,45198,T,A(0)T(657)C(1)G(0),301,658,.,99.8480243161094,0.151975683890578,.,...,.,.,.,T,.,.,.,T,T,T
996,B73V4_ctg129,45233,A,A(279)T(0)C(139)G(0),541,418,66.7464114832536,.,33.2535885167464,.,...,.,.,.,A,.,.,.,A,.,C
997,B73V4_ctg129,45240,A,A(274)T(0)C(104)G(0),581,378,72.4867724867725,.,27.5132275132275,.,...,.,.,.,A,.,.,.,A,.,C
998,B73V4_ctg129,45257,A,A(286)T(0)C(0)G(200),473,486,58.8477366255144,.,.,41.1522633744856,...,.,.,.,A,.,.,.,A,A,G


The imputed matrix has 979873 rows × 953 columns. It's structure is different than the one from the raw matrix. Nonetheless, the chromosome and position information is still available.

Keep in mind that some nucleotides are reported as `N`.

> That is normal. When the sequencing call is ambiguous (like maybe they read multiple possible conflicting nucleotides at that position even though these are inbreds, or the read quality was really low, etc) they use `N` to denote this. Full imputation would convert these to a specific call. So, even though this file has been imputed, it is only a partial imputation to fill in completely missing scores but not ambiguous ones.

In [7]:
snp_matrix_imputed

,rs,alleles,chrom,pos,strand,assembly,center,protLSID,assayLSID,panel,...,YANG,YE_4,YE-CHI-HUNG,YELLOW_3-4,YING-55,Yong_28,Yu796_NS,ZS01250,ZS1791,ZS635
0,rs1_44306,NaN,1,44306,NaN,NaN,NaN,NaN,NaN,NaN,...,G,G,G,G,G,G,G,G,G,G
1,rs1_44441,NaN,1,44441,NaN,NaN,NaN,NaN,NaN,NaN,...,C,A,A,A,A,A,A,A,A,A
2,rs1_44879,NaN,1,44879,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C
3,rs1_45948,NaN,1,45948,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,T
4,rs1_46521,NaN,1,46521,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,T,T,T,T,T,T,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899779,rs10_150833697,NaN,10,150833697,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C
899780,rs10_150833709,NaN,10,150833709,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,T,T,T,T,T,T,T,T
899781,rs10_150833722,NaN,10,150833722,NaN,NaN,NaN,NaN,NaN,NaN,...,G,G,G,G,G,G,G,G,G,G
899782,rs10_150833724,NaN,10,150833724,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,T,T,T,T,T,T,T,T


Discard the chromosomes `11` and `12`. From the README (emphasis added):
> Imputed SNP calls B73 plus B73-derived novel transcripts. SNPs on contigs __that are not__ part of the 10 chromosomes are coded as being on chromosome 11, and SNPs on RTAs are coded as being on chromosome 12.
	widiv_942g_column_name_converter.txt - Converts genotype names between unimputed (column 'Original') and imputed (column 'NoSpaces').

In [8]:
if not clean:
    downIndex = np.array([])
    for i in range(1,11):
        chromo = snp_matrix_imputed[snp_matrix_imputed['chrom'] == i]
        downIndex = np.concatenate((downIndex, chromo.index.values))

    snp_matrix = snp_matrix_imputed.iloc[downIndex]

In [9]:
if not clean:
    snp_matrix

We now have 899784 rows × 953 columns in our cleaned matrix. Observe the math.

In [10]:
if not clean:
    print('The cleaned matrix has', snp_matrix_imputed.shape[0] - snp_matrix.shape[0], 
          'rows less than the original imputed matrix.')
    print('Observe that the last entry of the cleaned matrix has index', snp_matrix.index[-1],
         'while we have a total of', snp_matrix.shape[0], 'rows.')
    print('Thus, for future reference, we only need to read the first', snp_matrix.shape[0], 
          'rows from the original matrix.')

Alternatively, we can read the first 10 individual SNPs from the source file.

In [11]:
if not clean:
    sample_snps = []
    with open(src+file_snp_imp) as f:
        for i in range(11):
            sample_snps.append(f.readline())

    # Column names
    print(sample_snps[0])

    # First SNP
    print(sample_snps[1])

    # Tenth SNP
    print(sample_snps[10])

Mask the 10th chromosome to have a smaller dataset to work with. The `rs` column can be reconstructed from the position column. We can also drop the blank columns. 

> That's because [the original matrix] is a hmp file (hapmap) and those columns are required to be there even though they are seldom if ever used.

Now we only have 61462 rows to work with.

In [12]:
downIndex = np.array([])
for i in range(10,11):
    chromo = snp_matrix_imputed[snp_matrix_imputed['chrom'] == i]
    downIndex = np.concatenate((downIndex, chromo.index.values))

downIndex.astype(int)
snp_sample = snp_matrix_imputed.iloc[downIndex]
snp_sample = snp_sample.drop(columns=['rs','alleles', 'strand', 
                                      'assembly','center', 'protLSID', 
                                      'assayLSID','panel', 'QCcode'])

In [13]:
snp_sample

,chrom,pos,764,779,787,790,793,904,911,912,...,YANG,YE_4,YE-CHI-HUNG,YELLOW_3-4,YING-55,Yong_28,Yu796_NS,ZS01250,ZS1791,ZS635
838322,10,130347,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
838323,10,130350,C,C,T,C,C,T,T,T,...,T,T,T,T,T,T,T,T,C,T
838324,10,130488,G,G,C,G,G,C,C,C,...,C,C,C,C,C,C,C,C,G,C
838325,10,130523,T,C,C,T,T,C,C,C,...,C,C,C,C,C,C,C,C,T,C
838326,10,130656,T,T,C,T,T,C,C,C,...,C,C,C,C,C,C,C,C,T,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899779,10,150833697,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
899780,10,150833709,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T
899781,10,150833722,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
899782,10,150833724,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T


Save the reduced matrix. Not as small as I expected but not terribly large either. Alternatively, we can further reduce the matrix to consider only the first 1000 SNPs in the 10th chromosome. This downsampled data can be comfortably loaded in any spreadsheet editor (Gnumeric, LibreCalc, MS Excel, etc).

If you couldn't load the massive matrix at the very beginning, you can __look for `snp_imputed_chr10_sample.csv` in the `sample_data` directory in git!!!__

In [39]:
snp_sample.to_csv(dst+'snp_imputed_chr10.csv', index=False)
snp_sample.iloc[27000:32000].to_csv(dst+'snp_imputed_chr10_sample.csv', index=False)

Now let's explore the expression data!!

We have 44259 rows × 947 columns in total.
- Rows: FPKM for the same gene across all 942 cultivars
- Columns: 5 columns of metadata + 942 cultivars

In [15]:
genes_w_features_raw = pd.read_table(src+file_gen)
genes_w_features_raw

/home/ejam/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,gene,chromosome,feature_type,position_left,position_right,LH128,DKMBZA,CQ806,DKF274,Ill.Hy,...,NC328,NC326,PHV53,DKIBC2,A641,WIL900,Va22,E8501,PHP85,Oh43
0,Zm00001d012719,8,gene,179164454,179168169,5.825000,4.845710,6.323400,6.999010,6.661100,...,15.154700,12.870700,8.81507,13.77870,8.31790,12.78320,4.94950,9.39900,10.41090,15.840500
1,Zm00001d024742,10,gene,85863323,85863746,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
2,Zm00001d007137,2,tRNA_gene,223153553,223153627,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000
3,Zm00001d025653,10,gene,125107973,125113328,9.350780,9.954800,7.735490,10.360900,9.072890,...,4.685600,3.870230,5.27849,5.17829,3.80291,6.44675,5.61858,3.98055,4.17545,4.500850
4,Zm00001d036003,6,gene,65322472,65323171,8.955580,0.333921,5.445820,0.332438,1.232330,...,0.232934,0.721774,2.95424,8.44716,12.33680,1.19629,1.71452,7.01768,11.75760,0.793672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44254,Zm00001d028653,1,gene,41833737,41838204,368.777000,301.349000,412.319000,310.820000,278.737000,...,172.293000,186.552000,147.63300,165.60800,147.58300,244.82100,171.15400,219.07400,147.01700,165.612000
44255,Zm00001d009511,8,gene,68418436,68421217,6.003590,5.297160,10.421700,6.304130,4.856300,...,4.120700,4.670860,3.47752,4.92093,6.26904,4.95873,7.25625,5.66864,4.28185,5.765310
44256,Zm00001d032828,1,gene,239455289,239464371,151.479000,46.479600,65.075600,70.169900,59.555600,...,100.249000,102.226000,100.29500,64.90400,91.17440,101.80300,108.58200,63.72140,95.96760,56.404900
44257,Zm00001d002031,2,gene,4673452,4674247,0.237774,0.386886,0.179111,0.415566,0.192793,...,0.000000,0.000000,0.00000,0.00000,0.00000,1.51552,0.00000,0.00000,0.00000,2.555730


Keep only information of the genes in chromosomes. Ignore the contigs. 

Turns out the contigs are mingled with the chromosomes and we can't define a nice cutoff while reading. We must read the whole file and then filter, as below.

In [16]:
downIndex = np.array([])
for i in range(1,11):
    chromo = genes_w_features_raw[genes_w_features_raw['chromosome'] == '{}'.format(i)]
    downIndex = np.concatenate((downIndex, chromo.index.values))

downIndex.astype(int)
gene_w_features = genes_w_features_raw.iloc[downIndex].astype({'chromosome': int})

In [17]:
gene_w_features

,gene,chromosome,feature_type,position_left,position_right,LH128,DKMBZA,CQ806,DKF274,Ill.Hy,...,NC328,NC326,PHV53,DKIBC2,A641,WIL900,Va22,E8501,PHP85,Oh43
7,Zm00001d033979,1,gene,279975978,279978430,76.133600,65.408400,38.507300,52.069200,75.549300,...,42.306200,41.304400,36.242400,33.992500,30.771400,23.95840,24.665400,20.311500,17.770000,38.619700
22,Zm00001d031073,1,gene,175980668,175983357,1.381800,0.621732,0.852335,1.681980,0.958799,...,1.341690,1.241070,0.877111,1.106620,1.758750,2.21258,1.086100,1.034650,2.463450,0.652187
23,Zm00001d034961,1,gene,306881093,306895296,0.026991,0.005325,0.016174,0.011593,0.018716,...,0.009031,0.000000,0.015687,0.030662,0.000000,0.00000,0.019362,0.018610,0.034208,0.026505
35,Zm00001d022701,1,lincRNA_gene,27303532,27304500,0.000000,0.040027,0.018941,0.088167,0.060270,...,0.000000,0.000000,0.000000,0.181800,0.110069,0.14295,0.111420,0.110546,0.000000,0.074827
42,Zm00001d022951,1,lincRNA_gene,247966116,247966918,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43951,Zm00001d023459,10,gene,6289475,6298739,7.743020,12.641800,10.975500,12.337900,15.497200,...,7.251110,13.711400,15.806400,16.518300,6.885620,22.86330,8.189910,6.772000,7.192290,10.332200
43976,Zm00001d026184,10,gene,140529821,140530718,0.437610,0.361114,0.366553,0.512719,0.149871,...,0.742883,0.587186,2.192500,0.110062,1.083900,1.57213,0.272518,0.154927,0.975337,0.605777
43984,Zm00001d024000,10,gene,35675291,35679293,38.797700,28.401200,3.048510,4.423800,8.698070,...,0.093581,2.816500,5.026650,3.186000,0.373411,18.87170,0.000000,0.000000,9.644770,33.329400
44023,Zm00001d025251,10,gene,111149588,111152196,0.081500,0.000000,0.068706,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.046316,0.062825,0.00000,0.000000,0.252784,0.000000,0.033119


In [18]:
print('We discarded', genes_w_features_raw.shape[0] - gene_w_features.shape[0],'rows.')

We discarded 584 rows.


Downsample the matrix. Consider only the genes in the 10th chromosome.

In [20]:
gene_sample = gene_w_features[gene_w_features['chromosome'] == 10]
gene_sample

,gene,chromosome,feature_type,position_left,position_right,LH128,DKMBZA,CQ806,DKF274,Ill.Hy,...,NC328,NC326,PHV53,DKIBC2,A641,WIL900,Va22,E8501,PHP85,Oh43
1,Zm00001d024742,10,gene,85863323,85863746,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Zm00001d025653,10,gene,125107973,125113328,9.350780,9.954800,7.735490,10.360900,9.072890,...,4.685600,3.870230,5.278490,5.178290,3.802910,6.446750,5.618580,3.980550,4.175450,4.500850
6,Zm00001d024903,10,gene,93908316,93911235,0.954312,0.335773,0.689938,0.106595,0.200660,...,1.809510,1.671710,2.940510,1.523060,7.885560,4.522500,1.478990,9.566990,6.826610,0.509826
11,Zm00001d023618,10,gene,12298031,12300071,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,Zm00001d024825,10,gene,89733155,89739279,0.857062,0.689824,0.613990,1.146330,1.137050,...,0.617702,0.317978,0.354413,0.412837,0.495563,0.312885,0.616760,0.450897,0.610228,0.472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43951,Zm00001d023459,10,gene,6289475,6298739,7.743020,12.641800,10.975500,12.337900,15.497200,...,7.251110,13.711400,15.806400,16.518300,6.885620,22.863300,8.189910,6.772000,7.192290,10.332200
43976,Zm00001d026184,10,gene,140529821,140530718,0.437610,0.361114,0.366553,0.512719,0.149871,...,0.742883,0.587186,2.192500,0.110062,1.083900,1.572130,0.272518,0.154927,0.975337,0.605777
43984,Zm00001d024000,10,gene,35675291,35679293,38.797700,28.401200,3.048510,4.423800,8.698070,...,0.093581,2.816500,5.026650,3.186000,0.373411,18.871700,0.000000,0.000000,9.644770,33.329400
44023,Zm00001d025251,10,gene,111149588,111152196,0.081500,0.000000,0.068706,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.046316,0.062825,0.000000,0.000000,0.252784,0.000000,0.033119


If you couldn't load the massive matrix at the very beginning, you can __look for `gene_w_features_chr10.csv` in the `sample_data` directory in git!!!__

In [38]:
gene_sample.to_csv(dst+'gene_w_features_chr10.csv', index=False)

In [36]:
val = 111166032
l_match = gene_sample[gene_sample['position_left'] <= val]
r_match = l_match[l_match['position_right'] >= val]
r_match

,gene,chromosome,feature_type,position_left,position_right,LH128,DKMBZA,CQ806,DKF274,Ill.Hy,...,NC328,NC326,PHV53,DKIBC2,A641,WIL900,Va22,E8501,PHP85,Oh43
33396,Zm00001d025252,10,gene,111156937,111166083,2.54496,2.37732,2.68808,3.28275,3.22185,...,2.3473,3.18874,2.98677,3.26416,2.44131,4.09,3.53631,3.1623,3.44806,2.47579


__To Do:__
- Match efficiently each SNP to its respective gene based on the position information.